<a href="https://colab.research.google.com/github/Ut-Niv/routes_on_map/blob/main/routes_on_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Showing routes on maps: a pragmatic approach
# Displaying Sites

# we will use the FOLIUM library to create maps and Geopy for geocoding

from typing import Tuple, List, Dict

import pandas as pd
import folium

pd.set_option('display.precision', 2) # for showing 2 decimal points

# Ceate DataFrame
df = pd.DataFrame([
     ['hotel',              48.8527, 2.3542],
     ['Sacre Coeur',        48.8867, 2.3431],
     ['Louvre',             48.8607, 2.3376],
     ['Montmartre',         48.8872, 2.3388],
     ['Port de Suffren',    48.8577, 2.2902],
     ['Arc de Triomphe',    48.8739, 2.2950],
     ['Av. Champs Élysées', 48.8710, 2.3036],
     ['Notre Dame',         48.8531, 2.3498],
     ['Tour Eiffel',        48.8585, 2.2945]
], columns = pd.Index(['Site', 'Langitude', 'Longitude'], name = 'Paris'))

# Our DataFrame
df.head()

Paris,Site,Langitude,Longitude
0,hotel,48.85,2.35
1,Sacre Coeur,48.89,2.34
2,Louvre,48.86,2.34
3,Montmartre,48.89,2.34
4,Port de Suffren,48.86,2.29


In [ ]:
# Plot these sites on map using Felium

# create a map
avg_location = df[['Langitude', 'Longitude']].mean()
map = folium.Map(location = avg_location, zoom_start = 13)

for site in df.itertuples():
  marker = folium.Marker(location=(site.Langitude, site.Longitude), tooltip = site.Site)
  marker.add_to(map)

map

In [ ]:
"""# Displaying Routes

"""

df_routes = df.copy()
df_routes.index.name = 'Visit Order'
df_routes

df_route_segments = df_routes.join(
    df_routes.shift(-1),
    rsuffix = '_next'
).dropna()
df_route_segments

map_paris = folium.Map(location=avg_location, zoom_start=13)

for stop in df_route_segments.itertuples():
    # marker for current stop
    marker = folium.Marker(location=(stop.Langitude, stop.Longitude),
                           tooltip=stop.Site)
    # line for the route segment connecting current to next stop
    line = folium.PolyLine(
        locations=[(stop.Langitude, stop.Longitude),
                   (stop.Langitude_next, stop.Longitude_next)],
        tooltip=f"{stop.Site} to {stop.Site_next}",
    )
    # add elements to the map
    marker.add_to(map_paris)
    line.add_to(map_paris)

# maker for last stop wasn't added in for loop, so adding it now
folium.Marker(location=(stop.Langitude_next, stop.Longitude_next),
              tooltip=stop.Site_next).add_to(map_paris);

map_paris

In [ ]:
"""# Enriching the map with the interactive information"""

# use of folium.Icon on each marker for distinct icon

map = folium.Map(location=avg_location, zoom_start = 13)
for stop in df_route_segments.itertuples():
  initial_stop = stop.Index == 0
  icon = folium.Icon(icon='home' if initial_stop else 'info-sign',
                     color='cadetblue' if initial_stop else 'red')
  marker = folium.Marker(location = (stop.Langitude, stop.Longitude),
                                     icon = icon, tooltip = stop.Site)
  line = folium.PolyLine(locations = [(stop.Langitude, stop.Longitude),
                                     (stop.Langitude_next, stop.Longitude_next)
                                    ],
                         tooltip = f"{stop.Site} to {stop.Site_next}",
                        )
  marker.add_to(map)
  line.add_to(map)


  folium.Marker(location=(stop.Langitude_next, stop.Longitude_next),
                tooltip = stop.Site_next,
                icon = folium.Icon(icon = 'info-sign', colot='red')
              ).add_to(map);
map

In [ ]:
"""# add distances between stops and the visit order of each stop."""

from geopy.distance import geodesic

_Location = Tuple[float, float]

def ellipsoidal_distance(point1: _Location, point2: _Location) -> float:
  # Calculate ellipsodial distance between point1 and point2
  return geodesic(point1, point2).meters

df_route_segments['distance_seg'] = df_route_segments.apply(
    lambda seg: ellipsoidal_distance((seg.Langitude, seg.Longitude),
                                     (seg.Langitude_next, seg.Longitude_next))
    ,axis = 1
)

# Distance between points distance_seg
df_route_segments

Paris,Site,Langitude,Longitude,Site_next,Langitude_next,Longitude_next,distance_seg
Visit Order,,,,,,,
0,hotel,48.85,2.35,Sacre Coeur,48.89,2.34,3867.74
1,Sacre Coeur,48.89,2.34,Louvre,48.86,2.34,2919.40
2,Louvre,48.86,2.34,Montmartre,48.89,2.34,2948.31
3,Montmartre,48.89,2.34,Port de Suffren,48.86,2.29,4844.92
4,Port de Suffren,48.86,2.29,Arc de Triomphe,48.87,2.29,1835.65
5,Arc de Triomphe,48.87,2.29,Av. Champs Élysées,48.87,2.30,708.53
6,Av. Champs Élysées,48.87,2.30,Notre Dame,48.85,2.35,3931.12
7,Notre Dame,48.85,2.35,Tour Eiffel,48.86,2.29,4102.26


In [ ]:
# Let's add stop numbers to the marker too
map = folium.Map(location=avg_location, zoom_start = 13)

for stop in df_route_segments.itertuples():
  initial_stop = stop.Index == 0
  icon = folium.Icon(icon='home' if initial_stop else 'info-sign',
                     color='cadetblue' if initial_stop else 'red')
  marker = folium.Marker(location = (stop.Langitude, stop.Longitude),
                                     icon = icon,
                        #  Display the name and stop number at each site's maker
                        tooltip = f"<b>Name</b>:{stop.Site} <br>" +
                                  f"<b>Stop Number</b>: {stop.Index}<br>"
                        )
  line = folium.PolyLine(locations = [(stop.Langitude, stop.Longitude),
                                     (stop.Langitude_next, stop.Longitude_next)
                                    ],
                                   tooltip=f"<b>From</b>: {stop.Site} <br>"
                                        + f"<b>To</b>: {stop.Site_next} <br>"
                                + f"<b>Distance</b>: {stop.distance_seg:.0f} m",
                         )

  marker.add_to(map)
  line.add_to(map)


  folium.Marker(location=(stop.Langitude_next, stop.Longitude_next),
                tooltip = f"<b>Name</b>: {stop.Site_next}<br>" +
                          f"<b>Stop number</b>: {stop.Index + 1} <br>",
                icon = folium.Icon(icon = 'info-sign', colot='red')
              ).add_to(map);
map

In [ ]:
# bit of HTML to render the “hover text” displayed on markers and lines more
# nicely.

def _make_routes_segments(df_route: pd.DataFrame) -> pd.DataFrame:
  df_route_segments = df_route.join(
      df_route.shift(-1),
      rsuffix = '_next').dropna()
  df_route_segments['distance_seg'] = df_route_segments.apply(
      lambda stop: ellipsoidal_distance(
          (stop.Langitude, stop.Longitude),
            (stop.Langitude_next, stop.Longitude_next)
        ), axis=1
      )
  return df_route_segments

def plot_route_on_map(df_route: pd.DataFrame) -> folium.Map:
  df_route_segments = _make_routes_segments(df_route)
  agv_location = df_route[['Langitude', 'Longitude']].mean()
  map_route = folium.Map(location = avg_location, zoom_start = 13)

  for stop in df_route_segments.itertuples():
    initial_stop = stop.Index == 0
    # marker for current stop
    icon = folium.Icon(icon='home' if initial_stop else 'info-sign',
                        color='cadetblue' if initial_stop else 'red')
    marker = folium.Marker(
        location=(stop.Langitude, stop.Longitude),
        icon=icon,
        tooltip=f"<b>Name</b>: {stop.Site} <br>" \
              + f"<b>Stop number</b>: {stop.Index} <br>"
    )
    # line for the route segment connecting current to next stop
    line = folium.PolyLine(
        locations=[(stop.Langitude, stop.Longitude),
                    (stop.Langitude_next, stop.Longitude_next)],
        # add to each line its start, end, and distance
        tooltip=f"<b>From</b>: {stop.Site} <br>" \
              + f"<b>To</b>: {stop.Site_next} <br>" \
              + f"<b>Distance</b>: {stop.distance_seg:.0f} m",
    )
    marker.add_to(map_route)
    line.add_to(map_route)

  first_stop = df_route_closed.iloc[0][['Site', 'Langitude', 'Longitude']]
  last_stop = df_route_closed.iloc[-1][['Site', 'Langitude', 'Longitude']]
  is_closed_tour = (first_stop == last_stop).all()

  if not is_closed_tour:
    folium.Marker(location = (stop.Langitude_next, stop.Longitude_next),
                  tooltip = f"<b>Name</b>: {stop.Site_next} <br>" +
                            f"<b>Stop number</b>: {stop.Index + 1} <br>",
                  icon = folium.Icon(icon = 'info-sign', color = 'red')
                ).add_to(map_route)
  return map_route

In [ ]:
"""# Dealing with closed routes, a.k.a. tours"""

df_route_closed = pd.concat(
    [df_routes, df_routes.head(1)], ignore_index = True
)
df_route_closed.index.name = df_routes.index.name
df_route_closed

plot_route_on_map(df_route_closed)